In [1]:
import numpy as np
import scipy.special

In [2]:
class BankAccount:
    def __init__(self, r):
        self.r = r

    def get_bank_account_value(self, t):
        return (1 + self.r) ** t


class MarketInstrument:
    def __init__(self, S_0):
        self.S_0 = S_0

    def calculate_call_option_value(self, market, T, K):
        return self._calculate_option_value(self.get_call_option_value, market, T, K)
        
    def calculate_put_option_value(self, market, T, K):
        return self._calculate_option_value(self.get_put_option_value, market, T, K)

    def _calculate_option_value(self, option_func_type, market, t, K):
        prices = [option_func_type(t, t-j, market.d, market.u, K) for j in range(0, t+1)]
        return self._get_value(t, prices, market.d, market.u, market.r, market.p)
    
    def _get_value(self, t, prices, d, u,r, p):
        if t == 0:
            return prices[0]
        else:
            for i in range(0, t):
                prices[i] =1/(1+r)*(p*prices[i] + (1-p)*prices[i+1])
            #print(prices[:t])
            return self._get_value(t-1, prices, d, u,r,  p)

    
    def get_call_option_value(self, t, j, d, u, K):
        return max(self.S_0 * u ** j * d ** (t - j) - K, 0)
    
    def get_put_option_value(self, t, j, d, u, K):
        return max(K - self.S_0 * u ** j * d ** (t - j), 0)
    


In [3]:
class CRRMarket:
    def __init__(self, S_0_values, d, u, r, T):
        self.B = BankAccount(r)
        self.p = (1 + r - d) / (u - d)
        self.S = [MarketInstrument(S_0_values[i]) for i in range(0, len(S_0_values))]
        self.d = d
        self.u = u
        self.r = r
        self.T = T
    
    def calculate_call_options_values(self, T, K):
        return [s.calculate_call_option_value(self, T, K[i]) for i, s in enumerate(self.S)]
    
    def calculate_put_options_values(self, T, K):
        return [s.calculate_put_option_value(self, T, K[i]) for i, s in enumerate(self.S)]

Market with more instruments example

In [4]:
market = CRRMarket([100, 10], 0.8, 1.3, 0.1, 10)
market.calculate_call_options_values(5, [90, 9])
market.calculate_put_options_values(5, [90, 9])

[2.6131834512048897, 0.2613183451204891]

Example Market from task: 

In [5]:
example_market = CRRMarket([100], 0.8, 1.3, 0.1, 10)
call_options = []
for i in range (0, 11):
    call_options.append(example_market.S[0].calculate_call_option_value(example_market,i, 90))
call_options

[10.0,
 21.81818181818182,
 29.05785123966944,
 35.718707738542456,
 41.84537121781301,
 46.730264375880964,
 51.952528772082964,
 55.743947875943654,
 60.18766988627156,
 63.454777835596246,
 66.97006486520782]

In [6]:
put_options = []
for i in range (0, 11):
    put_options.append(example_market.S[0].calculate_put_option_value(example_market,i, 90))
put_options

[0,
 3.6363636363636354,
 3.438016528925616,
 3.337039819684442,
 3.3165822006693486,
 2.6131834512048897,
 2.7551824769228896,
 1.928178516707216,
 2.1733341051475215,
 1.623563489119839,
 1.66896091386562]

## SPRAWDZENIE

In [7]:
def calculate_option_value_theory(option_func_type, market, t, K):
        scaling_value = 1 / market.B.get_bank_account_value(t)
        sum = 0
        for j in range(0, t+1):
            sum += scipy.special.binom(t, j) * market.p ** j * (1 - market.p) ** (t - j) * option_func_type(t, j, market.d, market.u, K)
        return sum * scaling_value


In [8]:
for i in range (0, 11):
    assert abs(calculate_option_value_theory(example_market.S[0].get_call_option_value, example_market,i, 90) - call_options[i]) < 000.1
    assert abs(calculate_option_value_theory(example_market.S[0].get_put_option_value, example_market,i, 90) - put_options[i]) < 000.1
print("Test passed.")

Test passed.


## ODPOWIEDŹ

In [9]:
print(f"Wycena opcji call: {call_options[10]} \nWycena opcji put: {put_options[10]}.")

Wycena opcji call: 66.97006486520782 
Wycena opcji put: 1.66896091386562.


Potwierdzam samodzielność powyższej pracy oraz niekorzystanie przeze mnie z niedozwolonych źródeł.
Stanisław Pawlak